In [16]:
import numpy as np
import matplotlib.pyplot as plt
import os
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))
import ROOT

In [2]:
# This is just a function to quickly save the output of a ROOT Histogram.
directory="XSEC_Tutorial_Outputs_Data/"
if not os.path.exists(directory):
    os.makedirs(directory)
def SAVEHIST(hist,filename,error=False):
    canv = ROOT.TCanvas('Can','Can',1000,642)
    if error:
        hist.Draw('HIST E')
    else:
        hist.Draw('HIST')
    canv.SaveAs(directory+filename+'.png')

In [34]:
#We're going to calculate our Number of Targest and Neutrino Flux First
######################################################################
rho_Ar = 1.3836; #density of argon in g*cm^-3                                                                      
V = (256.35 - 20.0) * (233.0 - 20.0) * (1036.8 - 20.0);#volume of detector cm: x*y*z                               
N_A = 6.023e+23; #avogadro's number                                                                                
m_mol = 39.95; #mass of argon in g*mol^-1                                                                          
N_targets = (rho_Ar*V*N_A)/m_mol; #number of target nuclei                                                         
print("Value of Volume: %E cm^3"%V)
print("Value of N_targets assuming 10cm FV border: %E N Targets"%N_targets)

"""
Input Neutrino Flux File contains:
  -The original histogram: hEnumu_cv
  -Histogram with the scaling factor applied: h_scaled
  -Histogram with scaling and gaus fit: h_scaled_w_fit
We need the h_scaled for purpose of xsec extraction
"""
in_flux = ROOT.TFile("root_files/pelee/Run_all/neutrino_flux.root")
Flux_h = in_flux.Get('h_scaled')
integral = Flux_h.Integral()
flux = 6.79E20 * integral
print(r'Value of the Flux: %E \nu cm^{-2}'%flux)

Value of Volume: 5.118830E+07 cm^3
Value of N_targets assuming 10cm FV border: 1.067769E+30 N Targets
Value of the Flux: 5.008460E+11 \nu cm^{-2}


In [33]:
#Here is where we grab the Efficiency Histograms and Smearing Matrices
#######################################################################
f_matrices = ROOT.TFile("root_files/pelee/Run_all/xsec_extraction.root")

var = ["_mom","_costheta","_phi"]
particles = ["_muon_all","_muon_contained","_muon_uncontainied","_lead_proton","_recoil\
_proton"]
other_var = ["_opening_angle_protons_lab","_opening_angle_protons_com","_opening_angle_mu_leadi\
ng","_opening_angle_mu_both","_delta_PT","_delta_alphaT","_delta_phiT","_nu_E"]

h_particles_eff = [] #list of eff for the particles
h_particles_matrices = [] #list of matrices for the particles
h_other_eff = [] #list of eff for the other variables
h_other_matrices = [] #list of matrices for the other variables

for i in range(0,len(particles)):
    for j in range(0,len(var)):
        h_particles_eff.append(f_matrices.Get("h_particle_num%s%s"%(particles[i],var[j])))
        h_particles_matrices.append(f_matrices.Get("h_particle_matrices%s%s_smearing"%(particles[i],var[j])))

for i in range(0,len(other_var)): 
    h_other_eff.append(f_matrices.Get("h_other_eff_num%s"%other_var[i]))
    h_other_matrices.append(f_matrices.Get("h_other_matrices%s_smearing"%other_var[i]))

In [38]:
#Now we are going to grab all the histograms that will consistute the numerator of our XSec:
##########################################################################################

#BNB Histograms
f_bnb = ROOT.TFile("root_files/pelee/Run_all/histograms_pelee_bnb.root")
h_bnb_particles = []
h_bnb_other = []

#EXT Histograms
f_ext = ROOT.TFile("root_files/pelee/Run_all/histograms_pelee_ext.root")
h_ext_particles = []
h_ext_other = []

#Dirt Histograms
f_dirt = ROOT.TFile("root_files/pelee/Run_all/histograms_pelee_dirt_wgt.root")
h_dirt_particles = []
h_dirt_other = []
 
#Overlay Histograms
f_overlay = ROOT.TFile("root_files/pelee/Run_all/histograms_pelee_overlay_wgt.root")
channel = ["_total", "_cc2p0pi","_ccNp1pi","_ccNp0pi","_cc1p0pi","_nc","_ccNpNpi",
            "_cc0p0pi","_other","_outfv","_ccnue"]
channel_raquel = ["_total", "_ccMEC", "_ccRES","_ccQE","_nc",
                   "_ccDIS","_ccCOH","_outfv","_other","_ccNue"]
h_overlay_particles_mine = [] #list of overlay histograms with my definitions: particles
h_overlay_other_mine = [] #list of overlay histograms with my definitions: other
h_overlay_particles_raquel = [] #list of overlay histograms with Raquel's: particles
h_overlay_other_raquel = [] #list of overlay histograms with Raquel's: other


#Now to Grab the histograms
for i in range(0,len(particles)):
    for j in range(0,len(var)):
        h_bnb_particles.append(f_bnb.Get("h%s%s_bnb"%(particles[i],var[j])))
        h_ext_particles.append(f_ext.Get("h%s%s_ext"%(particles[i],var[j])))
        h_dirt_particles.append(f_dirt.Get("h%s%s_ext"%(particles[i],var[j])))
        
        for k in range(0,len(channels)):
            h_overlay_particles_mine.append(f_overlay.Get("h%s%s%s"%(particles[i],var[j],channel[k])))
        for k in range (0, len(channels_raquel)):
            h_overlay_particles_raquel.append(f_overlay.Get("h%s%s%s"%(particles[i],var[j],channel[k])))
            
for i in range(0,len(other_var)): 
    h_bnb_other.append(f_bnb.Get("h_%s_bnb"%other_var[i]))
    h_ext_other.append(f_ext.Get("h_%s_bnb"%other_var[i]))
    h_dirt_other.append(f_dirt.Get("h_%s_bnb"%other_var[i]))

    for k in range(0,len(channels)):
        h_overlay_other_mine.append(f_overlay.Get("h_%s%s"%(other_var[i],channel[k])))
    for k in range (0, len(channels_raquel)):
        h_overlay_other_raquel.append(f_overlay.Get("h_%s%s"%(other_var[i],channel[k])))

In [3]:
# Alright, let's extract a cross section. 
# Latex Format for XSEC Formula:
#    \sigma_i = 22 \frac{\sum_{j} U(N_j-b_j)}{\epsilon_i\Phi_i N_t}
# 
#  Or, more explicitly:
#
# 
# The CrossSection in Energy Bin i is calculated by:
# 1) Taking the Data Events selected in reco energy bin j
# 2) Subtracting away the background events in reco energy bin j.
# 3) Take the remaining events, and 'unfold' (U) them in order
# to undo the true->reco energy smearing, while summing up contributions
# from all reco energy bins j. This is entirely handled by RooUnfold
# 4) Next divide by the efficiency of selecting the signal event in 
# true energy bin i to get the number of signal events that occurred, 
# not just the ones we found.
# 5) Divide by the (NuMu Flux)*(POT)*(Neutrons in Target) to get 
# Cross Section Per Neutron
# 6) Multiply by 22 neutrons/argon to get Cross Section per Argon Nucleus
# 7) Celebrate!!!

In [ ]:
"""
Steps 1 and 2: Subtract the Expected Background from the Data.
"""

#overlay background
h_overlay = h_overlay_other_mine[0].Clone("h_overlay%s",other_var[0])
h_overlay.Add(h_overlay_other_cc2p, -1)

#dirt background
h_dirt = h_dirt_other.Clone("h_dirt%s",other_var[0])

#total background
h_background = h_ext_other.Clone("h_ext%s",other_var[0])
h_background.Add(h_overlay)
h_background.Add(h_dirt)


h_bnb = h_bnb_other[0].Clone("h_bnb%s",other_var[0])
h_bnb.GetXaxis().SetTitle()
h_bnb.GetYaxis().SetTitle()


In [4]:
"""
Steps 1 and 2: Subtract the Expected Background from the Data.
This is done after the number of events has been scaled 
using tune weights and POT balancing
"""
Data_Minus_Background_h = ROOT.TH1D("Data_Minus_Background_h","Data_Minus_Background_h",15,0,1500)
Data_Minus_Background_h.SetXTitle("Reco Neutrino Energy")
Data_Minus_Background_h.SetYTitle("Selected Data Minus Background Events in 2.43e20")
for binx in range(Data_Minus_Background_h.GetNbinsX()):
    d = Data_h.GetBinContent(binx+1)
    b = Background_h.GetBinContent(binx+1)
    Data_Minus_Background_h.SetBinContent(binx+1,d-b)

NameError: name 'Data_h' is not defined